In [2]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
import feather
from fastai.text import *

from petfinder.data import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
path = 'C:\\work\\ML\\PetFinder\\'
bs=64

pets = get_data(isTest=False)
petsTest = get_data(isTest=True)

petsTest['AdoptionSpeed'] = 0

pets.AdoptionSpeed = pets.AdoptionSpeed.astype(float)

In [19]:
from fastai.tabular import *
from fastai.vision import *
from fastai.metrics import *
from fastai.text import *

dep_var = 'AdoptionSpeed'
cont_names, cat_names = cont_cat_split(pets, dep_var=dep_var)
procs = [FillMissing, Categorify, Normalize]
cat_names.remove('Filename')
cat_names.remove('PicturePath')
cat_names.remove('PetID')
cat_names.remove('Description')

In [20]:
cat_names, cont_names

(['Type',
  'Name',
  'Breed1',
  'Breed2',
  'Gender',
  'Color1',
  'Color2',
  'Color3',
  'MaturitySize',
  'FurLength',
  'Vaccinated',
  'Dewormed',
  'Sterilized',
  'Health',
  'State',
  'RescuerID',
  'NoImage',
  'NoDescription'],
 ['Age',
  'Quantity',
  'Fee',
  'VideoAmt',
  'PhotoAmt',
  'RescuerDogCount',
  'AvgSentenceSentimentMagnitude',
  'AvgSentenceSentimentScore',
  'SentimentMagnitude',
  'SentimentScore',
  'state_gdp',
  'state_population',
  'gdp_vs_population'])

In [21]:
byPetID = pets.groupby('PetID').size().reset_index()
byPetID = byPetID.sample(frac=.1).drop([0], axis=1)
byPetID['IsValidation'] = True
pets = pd.merge(pets, byPetID, how='left', on='PetID')
pets.IsValidation = pets.IsValidation.fillna(False)

In [29]:
data = (TabularList.from_df(pets, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_from_df(col='IsValidation')
                           .label_from_df(cols=dep_var, label_cls=FloatList)
                           .databunch())

In [30]:
kappa = KappaScore()
kappa.weights = "quadratic"
learn = tabular_learner(data, layers=[200,100], metrics=[rmse], y_range=[0, 4])
# learn.loss = MSELossFlat

learn = learn.to_fp16()

In [31]:
# learn.lr_find()
# learn.recorder.plot()

In [32]:
learn.fit_one_cycle(4, 1e-2)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.427018,1.196536,1.072282,01:22
1,0.217284,1.124343,1.037635,01:16
2,0.097555,1.137039,1.043076,01:16
3,0.048981,1.193035,1.066295,01:17


In [33]:
preds,y = learn.get_preds(ds_type=DatasetType.Valid)

In [35]:
preds = preds.numpy().round()

In [36]:
# preds = torch.softmax(preds, dim=1).argmax(1).numpy()

In [37]:
predictions = pets[pets.IsValidation == True]
predictions['Prediction'] = preds
predictions = predictions.groupby('PetID').mean().round()[['Prediction', 'AdoptionSpeed']]
preds, y = predictions['Prediction'], predictions['AdoptionSpeed']

In [38]:
quadratic_weighted_kappa(preds, y)

0.372985251366838

In [16]:
pets.shape

(58652, 33)

In [30]:
name='train'
p = pd.read_csv(name + '\\' + name + '.csv')

In [31]:
p.shape

(14993, 24)